In [ ]:
!pip install keras-tcn

In [ ]:
!pip install -U bnlp_toolkit

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,json,nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
path ='/content/drive/MyDrive/4.2/soft computing/Sentiment Analysis/bert multiclass classification/BanglaNewsCommentData.csv' 
stopwords_list = path+'bangla_stopwords.txt'
class color: # Text style
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/4.2/soft computing/Sentiment Analysis/bert multiclass classification/BanglaNewsCommentData.csv',encoding='UTF-8')


In [ ]:
data.head(15)

,Comment,Tag,cleaned,length,label
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন,7,0
1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,0,এটা কেন হচ্ছে সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে ...,14,0
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,0,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,11,0
3,চুরি নয় লুটপাট।,0,চুরি নয় লুটপাট,3,0
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,2,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,15,2
5,একটা ভাল প্রতিষ্ঠান কিভাবে লুট হয়ে যাচেছ!,0,একটা ভাল প্রতিষ্ঠান কিভাবে লুট হয়ে যাচেছ,7,0
6,সরকার যাদের এই ব্যাংকে নিয়গ দিয়েছে তারা ব্যাংক...,0,সরকার যাদের এই ব্যাংকে নিয়গ দিয়েছে তারা ব্যাংক...,21,0
7,পুরোপুরি আওয়ামীকরন হলেই আর এর অসতিতই থাকবেক না!,0,পুরোপুরি আওয়ামীকরন হলেই আর এর অসতিতই থাকবেক না,8,0
8,এককথায় ব্যাংকটিতে ডাকাতি হয়েছে এবং হতেই থাকবে।,0,এককথায় ব্যাংকটিতে ডাকাতি হয়েছে এবং হতেই থাকবে,7,0
9,ইসলামি ব্যাংক প্রারম্ভ থেকেই গ্রাহকদের পছন্দের...,0,ইসলামি ব্যাংক প্রারম্ভ থেকেই গ্রাহকদের পছন্দের...,29,0


In [ ]:
data.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
# Data cleaning function
def process_comments(Comment): 
    Comment = re.sub('[^\u0980-\u09FF]',' ',str(Comment)) #removing unnecessary punctuation
    return Comment

In [ ]:
data['cleaned'] = data['Comment'].apply(process_comments)  

In [ ]:
# Length of each Reveiws
data['length'] = data['cleaned'].apply(lambda x:len(x.split()))
# Remove the reviews with least words
dataset = data.loc[data.length>1]
dataset = dataset.reset_index(drop = True)
print("After Cleaning:","\nRemoved {} Small Reviews".format(len(data)-len(dataset)),
      "\nTotal Reviews:",len(dataset))

After Cleaning: 
Removed 25 Small Reviews 
Total Reviews: 13777


In [ ]:
sample_data = [10,100,25,854,966,1500,1589,1700,2000,2500,4000,4500]

In [ ]:
def label_encoding(sentiment,bool):
    """
    This function will return the encoded labels in array format. 
    
    Args:
        sentiment: series of class names(str)
        bool: boolean (True or False)
        
    Returns:
        labels: numpy array 
    """
    le = LabelEncoder()
    le.fit(sentiment)
    encoded_labels = le.transform(sentiment)
    labels = np.array(encoded_labels) # Converting into numpy array
    class_names =le.classes_ ## Define the class names again
    if bool == True:
        print("\n\t\t\t===== Label Encoding =====","\nClass Names:-->",le.classes_)
        for i in sample_data:
            print(sentiment[i],' ', encoded_labels[i],'\n')

    return labels


In [ ]:
data['label'] = label_encoding(data.Tag,True)

In [ ]:
dataset.cleaned = dataset.cleaned.apply(lambda x:x.replace('\n',' '))

In [ ]:
""" def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < lenght:
            max_length = length
    return max_length """

" def max_length(sequences):\n    '''\n    input:\n        sequences: a 2D list of integer sequences\n    output:\n        max_length: the max length of the sequences\n    '''\n    max_length = 0\n    for i, seq in enumerate(sequences):\n        length = len(seq)\n        if max_length < lenght:\n            max_length = length\n    return max_length "

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", data.cleaned[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(data.cleaned)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(data.cleaned)
max_len = 30 
#max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাসক হিসাবে জামাতের সাথেই আওয়ামী লীগ পারে না 
Into a sequence of int: [862, 148, 326, 200, 164, 34, 8, 5490, 486, 808, 2848, 130, 174, 45, 2]
Into a padded sequence: [ 862  148  326  200  164   34    8 5490  486  808 2848  130  174   45
    2    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [ ]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
না 2
করে 3
আর 4
কি 5
এই 6
হবে 7
হয় 8
জন্য 9
করা 10
33838


In [ ]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 4, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.05)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.05)(conc)
    outp = Dense(1, activation="tanh")(conc)   

    model = Model(inputs=inp, outputs=outp)
   
    
    return model

In [ ]:
import tensorflow as tf
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision() 

In [ ]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activation = 'relu'
kernel_size = 4

# Separate the sentences and the labels
sentences, labels = list(data.cleaned), list(data.label)

X_train,X_test,y_train,y_test = train_test_split(sentences,labels,train_size = 0.8,
                                                  test_size = 0.2,random_state =1432)

# Turn the labels into a numpy array
y_train = np.array(y_train)
y_test= np.array(y_test)

# encode data using
# Cleaning and Tokenization
#tfidf = TfidfVectorizer(ngram_range=(1,3),use_idf=True,tokenizer=lambda x: x.split())
#tokenizer.fit_transform(X_train)
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)


training_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

max_len = 100
#max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
word_index = tokenizer.word_index
vocab_size = len(word_index)+1

# Define the input shape

model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)
#model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
# Train the model
model.fit(Xtrain, y_train, batch_size=50, epochs=10, verbose=1,callbacks=[callbacks], validation_data=(Xtest, y_test))

# evaluate the model
loss, acc = model.evaluate(Xtest, y_test, verbose=0)
print('Test Accuracy: {}'.format(acc*100))


Epoch 1/100
221/221 [==============================] - 7s 21ms/step - loss: 0.6220 - accuracy: 0.2854 - val_loss: 0.6234 - val_accuracy: 0.2695
Epoch 2/100
221/221 [==============================] - 4s 19ms/step - loss: 0.6112 - accuracy: 0.2743 - val_loss: 0.6202 - val_accuracy: 0.2695
Epoch 3/100
221/221 [==============================] - 4s 19ms/step - loss: 0.6074 - accuracy: 0.2955 - val_loss: 0.6242 - val_accuracy: 0.2695
Epoch 4/100
221/221 [==============================] - 4s 19ms/step - loss: 0.5973 - accuracy: 0.3263 - val_loss: 0.6295 - val_accuracy: 0.4698
Epoch 5/100
221/221 [==============================] - 4s 19ms/step - loss: 0.4962 - accuracy: 0.5255 - val_loss: 0.6247 - val_accuracy: 0.4531
Epoch 6/100
221/221 [==============================] - 4s 19ms/step - loss: 0.2794 - accuracy: 0.6622 - val_loss: 0.6266 - val_accuracy: 0.4350
Epoch 7/100
221/221 [==============================] - 4s 19ms/step - loss: 0.1793 - accuracy: 0.7268 - val_loss: 0.6603 - val_accuracy: